In [33]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pandas as pd
spark = SparkSession.builder.getOrCreate();


In [34]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.13.0,org.apache.spark:spark-sql-kafka-0-10_2.11:2.13.0 pyspark-shell' 
from ast import literal_eval



In [35]:
# dfk = spark \
#   .readStream \
#   .format("kafka") \
#   .option("kafka.bootstrap.servers", "localhost:9092") \
#   .option("subscribe", "ValidatedRecords") \
#   .option("startingOffsets", "earliest") \
#   .load() 


#df.printSchema()
  

In [36]:
""" appName = "Kafka Examples"
master = "local"

spark = SparkSession.builder \
    .master(master) \
    .appName(appName) \
    .getOrCreate()

kafka_servers = "localhost:9092"

dfk = spark \
    .read \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_servers) \
    .option("subscribe", "ValidatedRecords") \
    .load """

' appName = "Kafka Examples"\nmaster = "local"\n\nspark = SparkSession.builder     .master(master)     .appName(appName)     .getOrCreate()\n\nkafka_servers = "localhost:9092"\n\ndfk = spark     .read     .format("kafka")     .option("kafka.bootstrap.servers", kafka_servers)     .option("subscribe", "ValidatedRecords")     .load '

In [37]:
filepath = "/home/gildas/Bureau/ECE/data/2Tour2022.csv"
tab = []
df = pd.read_csv(filepath,encoding="latin1", sep=";")
tab = df.columns.to_list()

names = []
i = 28
x = 28
while x < 35 : 
    names.append("_c"+str(i))
    i = i + 1
    x = x + 1

schematemp = tab + names
schema = T.StructType()
for x in schematemp: 
    schema.add(x, T.StringType(), True)

df = spark.read.option("encoding", "latin1").schema(schema).options(delimiter=";").csv(filepath)
df = df.withColumnRenamed("Code du b.vote","Code du b_vote")
df.show()



+-------------------+----------------------+--------------------------+-----------------------------+------------------+---------------------+--------------+--------+-----------+---------+-------+---------+------+------------+------------+----+----------+----------+--------+---------+---------+---------+----+------+--------+----+----------+----------+----+----+------+------+----+-----+-----+
|Code du département|Libellé du département|Code de la circonscription|Libellé de la circonscription|Code de la commune|Libellé de la commune|Code du b_vote|Inscrits|Abstentions|% Abs/Ins|Votants|% Vot/Ins|Blancs|% Blancs/Ins|% Blancs/Vot|Nuls|% Nuls/Ins|% Nuls/Vot|Exprimés|% Exp/Ins|% Exp/Vot|N°Panneau|Sexe|   Nom|  Prénom|Voix|% Voix/Ins|% Voix/Exp|_c28|_c29|  _c30|  _c31|_c32| _c33| _c34|
+-------------------+----------------------+--------------------------+-----------------------------+------------------+---------------------+--------------+--------+-----------+---------+-------+---------+----

In [38]:
delete = []

df = df.drop(df.colRegex("`^*%*`"))
for col in df.columns : 
  if "%" in col : delete.append(col)
  if "Libellé" in col : delete.append(col)

for y in delete :
  df = df.drop(y)

df = df.drop("_c28","_c29","_c33","_c34")  

df.show()




+-------------------+--------------------------+------------------+--------------+--------+-----------+-------+------+----+--------+---------+----+------+--------+----+------+------+----+
|Code du département|Code de la circonscription|Code de la commune|Code du b_vote|Inscrits|Abstentions|Votants|Blancs|Nuls|Exprimés|N°Panneau|Sexe|   Nom|  Prénom|Voix|  _c30|  _c31|_c32|
+-------------------+--------------------------+------------------+--------------+--------+-----------+-------+------+----+--------+---------+----+------+--------+----+------+------+----+
|Code du département|      Code de la circon...|Code de la commune|Code du b.vote|Inscrits|Abstentions|Votants|Blancs|Nuls|Exprimés|N°Panneau|Sexe|   Nom|  Prénom|Voix|  null|  null|null|
|                 01|                        04|               001|          0001|     643|        146|    497|    42|   5|     450|        1|   M|MACRON|Emmanuel| 237|LE PEN|Marine| 213|
|                 01|                        05|            

In [39]:
df = df.withColumn("Code", F.concat( F.col("Code du département"), F.lit("/"),\
                                     F.col("Code de la circonscription"), F.lit("/"),\
                                     F.col("Code de la commune"), F.lit("/"),\
                                     F.col("Code du b_vote")))\
       .withColumnRenamed("Code","Code1")

df = df.drop("Code du département","Code de la circonscription","Code de la commune","Code du b_vote")
df.show()

+--------+-----------+-------+------+----+--------+---------+----+------+--------+----+------+------+----+--------------------+
|Inscrits|Abstentions|Votants|Blancs|Nuls|Exprimés|N°Panneau|Sexe|   Nom|  Prénom|Voix|  _c30|  _c31|_c32|               Code1|
+--------+-----------+-------+------+----+--------+---------+----+------+--------+----+------+------+----+--------------------+
|Inscrits|Abstentions|Votants|Blancs|Nuls|Exprimés|N°Panneau|Sexe|   Nom|  Prénom|Voix|  null|  null|null|Code du départeme...|
|     643|        146|    497|    42|   5|     450|        1|   M|MACRON|Emmanuel| 237|LE PEN|Marine| 213|      01/04/001/0001|
|     213|         45|    168|    23|   3|     142|        1|   M|MACRON|Emmanuel|  94|LE PEN|Marine|  48|      01/05/002/0001|
|    1130|        322|    808|    71|  22|     715|        1|   M|MACRON|Emmanuel| 360|LE PEN|Marine| 355|      01/05/004/0001|
|    1129|        313|    816|    45|  18|     753|        1|   M|MACRON|Emmanuel| 426|LE PEN|Marine| 32

In [40]:
df0 = df.select("Code1","Nom","Prénom","Voix")
df1 = df.select("Code1","_c30","_c31","_c32")\
        .withColumnRenamed("_c30","Nom").withColumnRenamed("_c31","Prénom").withColumnRenamed("_c32","Voix")

dfnew = df0.union(df1)
dfnew.count()

139366

In [41]:
df = df.select("Code1","Inscrits","Abstentions","Votants","Blancs","Nuls","Exprimés").withColumnRenamed("Code1","Code")
df = df.join(dfnew, df.Code == dfnew.Code1 ,"inner").drop("Code1")
df.show()

+--------------------+--------+-----------+-------+------+----+--------+------+--------+----+
|                Code|Inscrits|Abstentions|Votants|Blancs|Nuls|Exprimés|   Nom|  Prénom|Voix|
+--------------------+--------+-----------+-------+------+----+--------+------+--------+----+
|Code du départeme...|Inscrits|Abstentions|Votants|Blancs|Nuls|Exprimés|   Nom|  Prénom|Voix|
|      01/04/001/0001|     643|        146|    497|    42|   5|     450|MACRON|Emmanuel| 237|
|      01/05/002/0001|     213|         45|    168|    23|   3|     142|MACRON|Emmanuel|  94|
|      01/05/004/0001|    1130|        322|    808|    71|  22|     715|MACRON|Emmanuel| 360|
|      01/05/004/0002|    1129|        313|    816|    45|  18|     753|MACRON|Emmanuel| 426|
|      01/05/004/0003|    1213|        303|    910|    87|  21|     802|MACRON|Emmanuel| 449|
|      01/05/004/0004|     932|        270|    662|    55|  21|     586|MACRON|Emmanuel| 313|
|      01/05/004/0005|    1046|        386|    660|    47|  

In [42]:
df = df.orderBy(F.col("Code"),F.col("Nom"))
df.show()

+--------------+--------+-----------+-------+------+----+--------+------+--------+----+
|          Code|Inscrits|Abstentions|Votants|Blancs|Nuls|Exprimés|   Nom|  Prénom|Voix|
+--------------+--------+-----------+-------+------+----+--------+------+--------+----+
|01/01/016/0001|     326|         59|    267|    12|   5|     250|LE PEN|  Marine| 138|
|01/01/016/0001|     326|         59|    267|    12|   5|     250|MACRON|Emmanuel| 112|
|01/01/024/0001|    1207|        229|    978|    49|  17|     912|LE PEN|  Marine| 420|
|01/01/024/0001|    1207|        229|    978|    49|  17|     912|MACRON|Emmanuel| 492|
|01/01/024/0002|    1247|        257|    990|    57|  14|     919|LE PEN|  Marine| 423|
|01/01/024/0002|    1247|        257|    990|    57|  14|     919|MACRON|Emmanuel| 496|
|01/01/029/0001|     445|         79|    366|    14|   6|     346|LE PEN|  Marine| 199|
|01/01/029/0001|     445|         79|    366|    14|   6|     346|MACRON|Emmanuel| 147|
|01/01/038/0001|     594|       

In [43]:
df = df.withColumn("Tour", lit(2))\
       .withColumn('Code du département', F.split(F.col("Code"), '/').getItem(0)) \
       .withColumn('Code de la circonscription', F.split(F.col("Code"), '/').getItem(1)) \
       .withColumn('Code de la commune', F.split(F.col("Code"), '/').getItem(2))\
       .withColumn('Code du b_vote', F.split(F.col("Code"), '/').getItem(3))\
       .drop("Code")

df = df.select("Tour","Code du département","Code de la circonscription","Code de la commune","Code du b_vote",\
          "Inscrits","Abstentions","Votants","Blancs","Nuls","Exprimés","Nom","Prénom","Voix")

df.show()

+----+-------------------+--------------------------+------------------+--------------+--------+-----------+-------+------+----+--------+------+--------+----+
|Tour|Code du département|Code de la circonscription|Code de la commune|Code du b_vote|Inscrits|Abstentions|Votants|Blancs|Nuls|Exprimés|   Nom|  Prénom|Voix|
+----+-------------------+--------------------------+------------------+--------------+--------+-----------+-------+------+----+--------+------+--------+----+
|   2|                 01|                        01|               016|          0001|     326|         59|    267|    12|   5|     250|LE PEN|  Marine| 138|
|   2|                 01|                        01|               016|          0001|     326|         59|    267|    12|   5|     250|MACRON|Emmanuel| 112|
|   2|                 01|                        01|               024|          0001|    1207|        229|    978|    49|  17|     912|LE PEN|  Marine| 420|
|   2|                 01|                    

In [ ]:
df.write.csv("/home/gildas/Bureau/ECE/data/datatour2")